In [9]:
import numpy as np
import einops

In [43]:

def Cross(A, e, max_iters=1000):
  m, n = A.shape
  I = [i for i in range(m)]
  J = [j for j in range(n)]

  r = 0
  #maximum element
  i,j = np.unravel_index(np.argmax(np.abs(A), axis=None), A.shape)
  #начальные U V
  U = np.copy(A[i, :]).reshape(1, n)
  V = np.copy(A[:, j] / A[i, j]).reshape(m, 1)
  Err = A - V @ U
  #drop row i and col j from I and J
  I[i] = -1
  J[j] = -1
  i = 0
  j = 0
  iter = 0
  while True:
    iter += 1
    r += 1
    #find new max row and col
    for k in range(n):
      if (abs(Err[i,k]) > abs(Err[i,j])) and (J[k] != -1):
          j = k
    for k in range(n):
      if (abs(Err[k,j]) > abs(Err[i,j])) and (I[k] != -1):
          i = k
    tmp = abs(Err[i][j]) *((m-r)*(n-r))**0.5
    norm = np.linalg.norm(V @ U, ord='fro')
    if (tmp <= e*norm):
      r -= 1
      I = [x for x in I if x != -1]
      J = [x for x in J if x != -1]
      return A[I, :], A[I, J], A[:, J]
    elif iter > max_iters:
      I = [x for x in I if x != -1]
      J = [x for x in J if x != -1]
      return A[I, :], A[I, J], A[:, J]

    #обновляем U, V
    U = np.concatenate((U, Err[i, :].reshape(1, n)), axis = 0)
    V = np.concatenate((V, (Err[:, j] / Err[i,j]).reshape(m, 1)), axis = 1)
    I[i] = -1
    J[j] = -1
    Err = A - V @ U

In [44]:
def F_norm(tensor):
    return np.sum(tensor ** 2)

def truncatedSVD(A, delta):
    u, s, v = np.linalg.svd(A)
    rank = (np.abs(s) >= delta).sum()
    return u[:, :rank], s[:rank], v[:rank, :]

def numel(tensor):
    return tensor.size

In [49]:
"""
    In development
"""
def TTCross(tensor, eps):

    d = len(tensor.shape)
    delta = eps / np.sqrt(d - 1) * F_norm(tensor)
    Ctens = tensor.copy()
    
    rk = 1
    r_prev = 1

    G_list = []
    for k in range(1, d):
        Ctens = Ctens.reshape(
            r_prev * tensor.shape[k - 1], 
            numel(Ctens) // (r_prev * tensor.shape[k - 1])
        )
        #u, s, v = truncatedSVD(Ctens, delta)
        u, s, v = Cross(Ctens, delta, 100)
        print(u.shape, s.shape, v.shape)
        rk = s.shape[0]
        if k > 1:
            Gk = u.reshape(r_prev, tensor.shape[k - 1], rk)
        else:
            Gk = u.reshape(tensor.shape[k - 1], rk)
        G_list.append(Gk)
        r_prev = rk
        Ctens =  s @ v
    G_list.append(Ctens)
    #G_list.append(Ctens.reshape(r_prev, Ctens.shape[1] // tensor.shape[-1], tensor.shape[-1]))
    return G_list
    

In [50]:
from PIL import Image
import numpy as np

# Загружаем jpg картинку при помощи PIL
image = Image.open('amanita.jpg')

# Переводим картинку в массив numpy
image_array = np.array(image)
image_array = einops.reduce(image_array[1:, 1:, :].astype('float'), "(h 4) (w 4) c -> h w c", 'mean')
"""[1:, 1:]
image_array = (image_array - image_array.min()) / (image_array.max() - image_array.min())
image_array = einops.reduce(image_array, "(h 4) (w 4) c -> h w c", 'mean')"""

# Проверяем размерность массива
print(image_array.shape)

(56, 56, 3)


In [51]:
import matplotlib.pyplot as plt